In [7]:
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model

In [8]:
frame_width = 640
frame_height = 480
brightness = 120
threshold = 0.00
font = cv2.FONT_HERSHEY_SIMPLEX

In [9]:
cap = cv2.VideoCapture(0)
cap.set(3, frame_width)
cap.set(4, frame_height)
cap.set(10, brightness)

True

In [10]:
# model = load_model("garbage_with_preprocessing_model.h5")
model = load_model("people_garbage_with_preprocessing_model.h5")

In [11]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img


def getClassName(class_no):
    classes = [
        "Not Littering",
        "Littering",
    ]
    return classes[class_no] if 0 <= class_no < len(classes) else "Unknown"

In [12]:
while True:

    success, img_original = cap.read()

    if not success:
        break

    img = np.asarray(img_original)
    # img = cv2.resize(img, (128, 128))
    img = cv2.resize(img, (72, 128))
    img = preprocessing(img)

    cv2.imshow("Processed Image", img)
    # img = img.reshape(1, 128, 128, 1)
    img = img.reshape(1, 72, 128, 1)

    cv2.putText(
        img_original, "Class: ", (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA
    )
    cv2.putText(
        img_original, "Score: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA
    )

    predictions = model.predict(img)
    class_index = np.argmax(predictions)
    probability_value = np.amax(predictions)

    if probability_value > threshold:
        label = str(getClassName(class_index))
        score = str(round(probability_value, 2))

        cv2.putText(
            img_original,
            f"Class: {label}",
            (20, 35),
            font,
            0.75,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
        cv2.putText(
            img_original,
            f"Score: {score}",
            (20, 75),
            font,
            0.75,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
    cv2.imshow("Result", img_original)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()